#### 1. How to get the data from S3, impliment it practically and how to use mountpoint and get the data from S3 to databricks…impliment with 2 - 3 scenarios with sample example?

In [0]:
# Scenario - 1 (Direct Access)

df = spark.read.format("csv") \
    .option("header", "true") \
    .load("s3://abduldbtlearn/SamplePractical.csv")

df.show()


+-------+----------+-----------+----------+
|OrderID|CustomerID|OrderAmount| OrderDate|
+-------+----------+-----------+----------+
|   1001|      C001|      150.5|2023-08-10|
|   1002|      C002|       75.2|2023-08-11|
|   1003|      C001|        250|2023-08-12|
|   1004|      C003|         50|2023-08-13|
|   1005|      C002|     300.75|2023-08-14|
|   1006|      C004|      99.99|2023-08-15|
|   1007|      C003|      175.1|2023-08-16|
|   1008|      C005|        125|2023-08-17|
|   1009|      C001|         80|2023-08-18|
|   1010|      C006|        200|2023-08-19|
+-------+----------+-----------+----------+



In [0]:
# Senario - 2 (Mount S3 Bucket to Databricks)

ACCESS_KEY = dbutils.secrets.get(scope="testscope", key="aws-access-key")
SECRET_KEY = dbutils.secrets.get(scope="testscope", key="aws-secret-key")

AWS_BUCKET_NAME = "abduldbtlearn"
MOUNT_NAME = "/mnt/s3bucket"

ENCODED_SECRET_KEY = SECRET_KEY.replace("/", "%2F")

dbutils.fs.mount(
    source = f"s3a://{ACCESS_KEY}:{ENCODED_SECRET_KEY}@{AWS_BUCKET_NAME}",
    mount_point = MOUNT_NAME
)

print("S3 Bucket Mounted Successfully!")

S3 Bucket Mounted Successfully!


In [0]:
display(dbutils.fs.ls("/mnt/s3bucket"))

path,name,size,modificationTime
dbfs:/mnt/s3bucket/SamplePractical.csv,SamplePractical.csv,308,1740442681000
dbfs:/mnt/s3bucket/csv/,csv/,0,1740443111327
dbfs:/mnt/s3bucket/processed-data/,processed-data/,0,1740443111327


In [0]:
df = spark.read.format("csv").option("header", "true").load("dbfs:/mnt/s3bucket/SamplePractical.csv")
df.show()

+-------+----------+-----------+----------+
|OrderID|CustomerID|OrderAmount| OrderDate|
+-------+----------+-----------+----------+
|   1001|      C001|      150.5|2023-08-10|
|   1002|      C002|       75.2|2023-08-11|
|   1003|      C001|        250|2023-08-12|
|   1004|      C003|         50|2023-08-13|
|   1005|      C002|     300.75|2023-08-14|
|   1006|      C004|      99.99|2023-08-15|
|   1007|      C003|      175.1|2023-08-16|
|   1008|      C005|        125|2023-08-17|
|   1009|      C001|         80|2023-08-18|
|   1010|      C006|        200|2023-08-19|
+-------+----------+-----------+----------+



In [0]:
# Senario - 3 (Using IAM Role-Based Access)

df1 = spark.read.format("csv") \
    .option("header", "true") \
    .load("s3://abduldbtlearn/SamplePractical.csv")

df1.show()

+-------+----------+-----------+----------+
|OrderID|CustomerID|OrderAmount| OrderDate|
+-------+----------+-----------+----------+
|   1001|      C001|      150.5|2023-08-10|
|   1002|      C002|       75.2|2023-08-11|
|   1003|      C001|        250|2023-08-12|
|   1004|      C003|         50|2023-08-13|
|   1005|      C002|     300.75|2023-08-14|
|   1006|      C004|      99.99|2023-08-15|
|   1007|      C003|      175.1|2023-08-16|
|   1008|      C005|        125|2023-08-17|
|   1009|      C001|         80|2023-08-18|
|   1010|      C006|        200|2023-08-19|
+-------+----------+-----------+----------+



#### 2. Where will you check data skewness in DataBricks? Impliment dataskweness in databricks and write a code to handle that in databricks? Impliment with a scenario

In [0]:
from pyspark.sql.functions import *

# Creating a sample DataFrame with skewed data
data = [(1, "Store_A"), (2, "Store_B"), (3, "Store_C"), (4, "Store_A"), 
        (5, "Store_A"), (6, "Store_A"), (7, "Store_A"), (8, "Store_A"), 
        (9, "Store_B"), (10, "Store_A")] * 1000  # Skewing Store_A data

df2 = spark.createDataFrame(data, ["order_id", "store"])
df2.show()

+--------+-------+
|order_id|  store|
+--------+-------+
|       1|Store_A|
|       2|Store_B|
|       3|Store_C|
|       4|Store_A|
|       5|Store_A|
|       6|Store_A|
|       7|Store_A|
|       8|Store_A|
|       9|Store_B|
|      10|Store_A|
|       1|Store_A|
|       2|Store_B|
|       3|Store_C|
|       4|Store_A|
|       5|Store_A|
|       6|Store_A|
|       7|Store_A|
|       8|Store_A|
|       9|Store_B|
|      10|Store_A|
+--------+-------+
only showing top 20 rows



In [0]:
# Checking Data Skewness
df2.groupBy("store").count().orderBy(col("count").desc()).show()


+-------+-----+
|  store|count|
+-------+-----+
|Store_A| 7000|
|Store_B| 2000|
|Store_C| 1000|
+-------+-----+



In [0]:
# Adding a salt key
df2 = df2.withColumn("salt", floor(rand() * 5))
df2 = df2.withColumn("store_salted", concat(col("store"), lit("_"), col("salt")))
# df2.show()
df2.groupBy("store_salted").count().orderBy(col("count").desc()).show()

+------------+-----+
|store_salted|count|
+------------+-----+
|   Store_A_2| 1430|
|   Store_A_3| 1416|
|   Store_A_1| 1397|
|   Store_A_0| 1392|
|   Store_A_4| 1365|
|   Store_B_2|  409|
|   Store_B_0|  407|
|   Store_B_3|  406|
|   Store_B_4|  403|
|   Store_B_1|  375|
|   Store_C_3|  212|
|   Store_C_1|  207|
|   Store_C_2|  198|
|   Store_C_4|  192|
|   Store_C_0|  191|
+------------+-----+



In [0]:
# Creating a Reference Table

store_info_data = [
    ("Store_A", "Electronics"),
    ("Store_B", "Grocery"),
    ("Store_C", "Clothing")
]

store_info_df = spark.createDataFrame(store_info_data, ["store", "category"])


In [0]:
# Expand store_info_df to include all salt values
store_info_df = store_info_df.withColumn("salt", explode(array([lit(i) for i in range(5)])))
store_info_df = store_info_df.withColumn("store_salted", concat(col("store"), lit("_"), col("salt")))

# Perform the join on store_salted instead of store
final_df = df2.join(store_info_df, "store_salted", "left")

final_df.show()

+------------+--------+-------+----+-------+-----------+----+
|store_salted|order_id|  store|salt|  store|   category|salt|
+------------+--------+-------+----+-------+-----------+----+
|   Store_A_1|       5|Store_A|   1|Store_A|Electronics|   1|
|   Store_A_1|      10|Store_A|   1|Store_A|Electronics|   1|
|   Store_C_2|       3|Store_C|   2|Store_C|   Clothing|   2|
|   Store_A_0|       1|Store_A|   0|Store_A|Electronics|   0|
|   Store_A_2|       4|Store_A|   2|Store_A|Electronics|   2|
|   Store_A_2|       5|Store_A|   2|Store_A|Electronics|   2|
|   Store_A_2|       6|Store_A|   2|Store_A|Electronics|   2|
|   Store_A_0|       7|Store_A|   0|Store_A|Electronics|   0|
|   Store_A_0|       8|Store_A|   0|Store_A|Electronics|   0|
|   Store_A_2|      10|Store_A|   2|Store_A|Electronics|   2|
|   Store_A_2|       7|Store_A|   2|Store_A|Electronics|   2|
|   Store_A_0|       8|Store_A|   0|Store_A|Electronics|   0|
|   Store_A_2|       1|Store_A|   2|Store_A|Electronics|   2|
|   Stor

In [0]:
final_df.rdd.getNumPartitions()

4

In [0]:
final_df.rdd.glom().map(len).collect()

[2048, 3072, 2048, 2832]

#### 4. How to implement scd2 in databricks 

In [0]:
from pyspark.sql.functions import *

data = [
    (1, "Alice", "NY", "Gold"),
    (2, "Bob", "CA", "Silver"),
    (3, "Charlie", "TX", "Gold"),
    (4, "David", "FL", "Platinum")  # New customer
]

columns = ["customer_id", "name", "state", "membership"]

customers_source = spark.createDataFrame(data, columns)
customers_source.show()

+-----------+-------+-----+----------+
|customer_id|   name|state|membership|
+-----------+-------+-----+----------+
|          1|  Alice|   NY|      Gold|
|          2|    Bob|   CA|    Silver|
|          3|Charlie|   TX|      Gold|
|          4|  David|   FL|  Platinum|
+-----------+-------+-----+----------+



In [0]:
data = [
    (1, "Alice", "NY", "Silver", "2024-01-01", "9999-12-31", "Y"),
    (2, "Bob", "CA", "Silver", "2024-01-01", "9999-12-31", "Y"),
    (3, "Charlie", "TX", "Platinum", "2024-01-01", "9999-12-31", "Y")
]

columns = ["customer_id", "name", "state", "membership", "start_date", "end_date", "is_active"]

customers_dim = spark.createDataFrame(data, columns)
customers_dim.show()


+-----------+-------+-----+----------+----------+----------+---------+
|customer_id|   name|state|membership|start_date|  end_date|is_active|
+-----------+-------+-----+----------+----------+----------+---------+
|          1|  Alice|   NY|    Silver|2024-01-01|9999-12-31|        Y|
|          2|    Bob|   CA|    Silver|2024-01-01|9999-12-31|        Y|
|          3|Charlie|   TX|  Platinum|2024-01-01|9999-12-31|        Y|
+-----------+-------+-----+----------+----------+----------+---------+



In [0]:
from delta.tables import DeltaTable
from pyspark.sql.functions import col, current_date

# Convert customers_dim to a Delta table
customers_dim.write.format("delta").mode("overwrite").saveAsTable("customers_dim")

# Load the Delta table
dim_table = DeltaTable.forName(spark, "customers_dim")

# Define merge condition based on customer_id
merge_condition = "dim.customer_id = src.customer_id AND dim.is_active = 'Y'"

# Perform SCD Type 2 Merge
dim_table.alias("dim").merge(
    customers_source.alias("src"),
    merge_condition
).whenMatchedUpdate(
    condition="dim.membership <> src.membership OR dim.state <> src.state",  # Check for changes
    set={
        "end_date": current_date(),
        "is_active": lit("N")  # Expire the old record
    }
).whenNotMatchedInsert(
    values={
        "customer_id": "src.customer_id",
        "name": "src.name",
        "state": "src.state",
        "membership": "src.membership",
        "start_date": current_date(),
        "end_date": lit("9999-12-31"),
        "is_active": lit("Y")
    }
).execute()


In [0]:
df = spark.sql("SELECT * FROM customers_dim ORDER BY customer_id, start_date")
df.show()

+-----------+-------+-----+----------+----------+----------+---------+
|customer_id|   name|state|membership|start_date|  end_date|is_active|
+-----------+-------+-----+----------+----------+----------+---------+
|          1|  Alice|   NY|    Silver|2024-01-01|2025-02-23|        N|
|          2|    Bob|   CA|    Silver|2024-01-01|9999-12-31|        Y|
|          3|Charlie|   TX|  Platinum|2024-01-01|2025-02-23|        N|
|          4|  David|   FL|  Platinum|2025-02-23|9999-12-31|        Y|
+-----------+-------+-----+----------+----------+----------+---------+



#### 5. Write a SQL query to calculate the total revenue by product category from a sales table. The table Sales has columns: product_id, category_id, sale_amount, and there is a Categories table with columns category_id and category_name.

In [0]:
%sql
CREATE TABLE IF NOT EXISTS Sales (
    product_id INT,
    category_id INT,
    sale_amount DECIMAL(10,2)
);

In [0]:
%sql
CREATE TABLE IF NOT EXISTS Categories (
    category_id INT PRIMARY KEY,
    category_name VARCHAR(255)
);


In [0]:
%sql
INSERT INTO Categories (category_id, category_name) VALUES 
(1, 'Electronics'),
(2, 'Clothing'),
(3, 'Furniture');

INSERT INTO Sales (product_id, category_id, sale_amount) VALUES 
(101, 1, 500.00),
(102, 1, 300.00),
(103, 2, 150.00),
(104, 2, 100.00),
(105, 3, 700.00);


num_affected_rows,num_inserted_rows
5,5


In [0]:
%sql
SELECT * FROM sales;

product_id,category_id,sale_amount
101,1,500.00
102,1,300.00
103,2,150.00
104,2,100.00
105,3,700.00


In [0]:
%sql
SELECT 
    c.category_name, 
    SUM(s.sale_amount) AS total_revenue
FROM Sales s
JOIN Categories c ON s.category_id = c.category_id
GROUP BY c.category_name
ORDER BY total_revenue DESC;


category_name,total_revenue
Electronics,800.00
Furniture,700.00
Clothing,250.00


Databricks visualization. Run in Databricks to view.

#### 8. If I want to user input from databricks how can I take that ?

In [0]:
dbutils.widgets.text("user_name", "Enter your name")  # Default value: "Enter your name"
user_name = dbutils.widgets.get("user_name")

print(f"Hello, {user_name}!")  # Display user input

Hello, Jawwad!


In [0]:
dbutils.widgets.dropdown("country", "USA", ["USA", "India", "UK", "Canada"])
selected_country = dbutils.widgets.get("country")

print(f"Selected Country: {selected_country}")


Selected Country: India


#### 10. How do you make sure you data is backuped and what are the procees you have used ?

In [0]:
%sql
describe history sales;

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
1,2025-02-23T09:46:20Z,6469192871093063,abduljawwad246@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(469147981753331),0222-141020-q6jp4fy0,0,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 5, numOutputBytes -> 1122)",null,Databricks-Runtime/15.4.x-aarch64-photon-scala2.12
0,2025-02-23T09:44:25Z,6469192871093063,abduljawwad246@gmail.com,CREATE TABLE,"Map(partitionBy -> [], clusterBy -> [], description -> null, isManaged -> true, properties -> {""delta.enableDeletionVectors"":""true""}, statsOnLoad -> false)",null,List(469147981753331),0222-141020-q6jp4fy0,null,WriteSerializable,true,Map(),null,Databricks-Runtime/15.4.x-aarch64-photon-scala2.12


In [0]:
%sql
SELECT * FROM sales VERSION AS OF 0;

product_id,category_id,sale_amount


In [0]:
df = spark.table("sales")  
df.write.format("csv").option("header", "true").mode("overwrite").save("s3://abduldbtlearn/csv/sales/")

#### 15. PySpark Program to Calculate Daily Revenue Per Product & Store in Delta Table

In [0]:
%sql
CREATE TABLE sales_transactions (
    transaction_date DATE COMMENT 'Date of the transaction',
    transaction_id STRING COMMENT 'Unique transaction identifier',
    product_id STRING COMMENT 'Product identifier',
    quantity INT COMMENT 'Quantity of products sold',
    price DECIMAL(10,2) COMMENT 'Price of the product'
)
USING DELTA
COMMENT 'Table storing sales transactions'
PARTITIONED BY (transaction_date);

In [0]:
%sql
INSERT INTO sales_transactions VALUES
    ('2024-02-01', 'TXN001', 'P001', 2, 19.99),
    ('2024-02-01', 'TXN002', 'P002', 1, 29.99),
    ('2024-02-01', 'TXN003', 'P003', 3, 9.99),
    ('2024-02-02', 'TXN004', 'P004', 5, 15.49),
    ('2024-02-02', 'TXN005', 'P002', 2, 29.99),
    ('2024-02-03', 'TXN006', 'P001', 1, 19.99),
    ('2024-02-03', 'TXN007', 'P005', 4, 12.99),
    ('2024-02-04', 'TXN008', 'P006', 2, 24.99),
    ('2024-02-04', 'TXN009', 'P007', 3, 17.49),
    ('2024-02-05', 'TXN010', 'P003', 5, 9.99),
    ('2024-02-06', 'TXN011', 'P008', 1, 34.99),
    ('2024-02-06', 'TXN012', 'P001', 3, 19.99),
    ('2024-02-07', 'TXN013', 'P002', 4, 29.99),
    ('2024-02-08', 'TXN014', 'P009', 2, 11.99),
    ('2024-02-08', 'TXN015', 'P005', 6, 12.99);


num_affected_rows,num_inserted_rows
15,15


In [0]:
%sql
SELECT * FROM sales_transactions LIMIT 10;

transaction_date,transaction_id,product_id,quantity,price
2024-02-01,TXN001,P001,2,19.99
2024-02-01,TXN002,P002,1,29.99
2024-02-01,TXN003,P003,3,9.99
2024-02-06,TXN011,P008,1,34.99
2024-02-06,TXN012,P001,3,19.99
2024-02-04,TXN008,P006,2,24.99
2024-02-04,TXN009,P007,3,17.49
2024-02-03,TXN006,P001,1,19.99
2024-02-03,TXN007,P005,4,12.99
2024-02-07,TXN013,P002,4,29.99


In [0]:
# Compute total revenue (quantity * price) at day level per product
revenue_df = sales_df.groupBy("transaction_date", "product_id") \
    .agg(sum(col("quantity") * col("price")).alias("total_revenue"))

# Show results
revenue_df.show()

+----------------+----------+-------------+
|transaction_date|product_id|total_revenue|
+----------------+----------+-------------+
|      2024-02-01|      P002|        29.99|
|      2024-02-01|      P003|        29.97|
|      2024-02-08|      P009|        23.98|
|      2024-02-01|      P001|        39.98|
|      2024-02-08|      P005|        77.94|
|      2024-02-06|      P001|        59.97|
|      2024-02-02|      P002|        59.98|
|      2024-02-02|      P004|        77.45|
|      2024-02-06|      P008|        34.99|
|      2024-02-04|      P006|        49.98|
|      2024-02-03|      P005|        51.96|
|      2024-02-04|      P007|        52.47|
|      2024-02-03|      P001|        19.99|
|      2024-02-07|      P002|       119.96|
|      2024-02-05|      P003|        49.95|
+----------------+----------+-------------+



In [0]:
%sql
SELECT TRANSACTION_DATE, PRODUCT_ID, SUM(PRICE * QUANTITY) AS TOTAL_REVENUE 
FROM sales_transactions 
GROUP BY 1, 2
ORDER BY 2;

TRANSACTION_DATE,PRODUCT_ID,TOTAL_REVENUE
2024-02-03,P001,19.99
2024-02-06,P001,59.97
2024-02-01,P001,39.98
2024-02-07,P002,119.96
2024-02-02,P002,59.98
2024-02-01,P002,29.99
2024-02-01,P003,29.97
2024-02-05,P003,49.95
2024-02-02,P004,77.45
2024-02-08,P005,77.94


Databricks visualization. Run in Databricks to view.

In [0]:
# Define Unity Catalog table name
table_name = "databricks_practicals_new.default.sales_revenue"

# Write the results to Unity Catalog directly
revenue_df.write.format("delta").mode("overwrite").saveAsTable(table_name)


In [0]:
%sql
SELECT * FROM databricks_practicals_new.default.sales_revenue WHERE transaction_date = '2024-02-2';


transaction_date,product_id,total_revenue
2024-02-02,P002,59.98
2024-02-02,P004,77.45


#### 16. Result count for each join


In [0]:
%sql
CREATE TABLE table1 (
    col INT
);


INSERT INTO table1 VALUES
    (1),
    (2),
    (3),
    (5),
    (NULL);

num_affected_rows,num_inserted_rows
5,5


In [0]:
%sql
CREATE TABLE table2 (
    col INT
);

INSERT INTO table2 VALUES
    (1),
    (1),
    (2),
    (4),
    (6);

num_affected_rows,num_inserted_rows
5,5


In [0]:
%sql
SELECT * FROM table1 INNER JOIN table2 ON table1.col = table2.col;


col,col
1,1
2,2
1,1


In [0]:
%sql
SELECT * FROM table1 LEFT JOIN table2 ON table1.col = table2.col;


col,col
1,1
2,2
3,null
5,null
null,null
1,1


In [0]:
%sql
SELECT * FROM table1 RIGHT JOIN table2 ON table1.col = table2.col;


col,col
1,1
1,1
2,2
null,6
null,4


In [0]:
%sql
SELECT * FROM table1 FULL OUTER JOIN table2 ON table1.col = table2.col;

col,col
1,1
2,2
null,null
3,null
5,null
1,1
null,6
null,4


In [0]:
%sql
SELECT * FROM table1 LEFT ANTI JOIN table2 ON table1.col = table2.col;

col
3
5
null


#### 22. Sort without built-in fucntion in ascending order 


In [0]:
l = [64, 25, 12, 22, 11, 1, 2, 44, 3, 122, 23, 34]

# Selection Sort Algorithm
for i in range(len(l)):
    min_index = i
    for j in range(i + 1, len(l)):
        if l[j] < l[min_index]:  # Find the smallest element
            min_index = j
    l[i], l[min_index] = l[min_index], l[i]  # Swap elements

print("Sorted List:", l)


Sorted List: [1, 2, 3, 11, 12, 22, 23, 25, 34, 44, 64, 122]


#### 24. If we are having 10k records with duplicates and 9k records after removing duplicates how can we identify that these are the correct records after removing duplicates

In [0]:
from pyspark.sql.types import *

# Sample data with duplicates
data = [
    (1, "Alice"),
    (2, "Bob"),
    (3, "Charlie"),
    (4, "David"),
    (5, "Eve"),
    (1, "Alice"),  # Duplicate
    (2, "Bob"),    # Duplicate
    (6, "Frank"),
    (7, "Grace"),
    (8, "Hannah"),
    (9, "Isaac"),
    (10, "Jack"),
    (10, "Jack")   # Duplicate
]

# Define schema
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True)
])

# Create DataFrame
sample_df = spark.createDataFrame(data, schema)

# Show DataFrame
sample_df.show()

+---+-------+
| id|   name|
+---+-------+
|  1|  Alice|
|  2|    Bob|
|  3|Charlie|
|  4|  David|
|  5|    Eve|
|  1|  Alice|
|  2|    Bob|
|  6|  Frank|
|  7|  Grace|
|  8| Hannah|
|  9|  Isaac|
| 10|   Jack|
| 10|   Jack|
+---+-------+



In [0]:
print(f"Total records before removing duplicates: {sample_df.count()}")

Total records before removing duplicates: 13


In [0]:
df_unique = sample_df.dropDuplicates()
df_unique.show()
print(f"Total records after removing duplicates: {df_unique.count()}")

+---+-------+
| id|   name|
+---+-------+
|  2|    Bob|
|  1|  Alice|
|  3|Charlie|
|  4|  David|
|  5|    Eve|
|  7|  Grace|
|  6|  Frank|
|  8| Hannah|
| 10|   Jack|
|  9|  Isaac|
+---+-------+

Total records after removing duplicates: 10


In [0]:
df_duplicates = sample_df.exceptAll(df_unique)
df_duplicates.show()

+---+-----+
| id| name|
+---+-----+
|  2|  Bob|
|  1|Alice|
| 10| Jack|
+---+-----+



#### 26. Where do you import python boto library



In [0]:
%pip install boto3

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import boto3

# Create an S3 client
s3_client = boto3.client("s3")

# List all S3 buckets
response = s3_client.list_buckets()

# Print bucket names
for bucket in response["Buckets"]:
    print(bucket["Name"])


abdul-demo-athena-ap-south-1
abdul-dms-bucket
abdul-snowflake-airflow
abdul-snowflake-data
abduldbtlearn
aws-glue-assets-727646464992-ap-southeast-1
aws-practical-boto-demo-14112024
aws-practical-demo-bucket-14112024
cf-templates-18fzstqfiv1wk-ap-south-1
databricks-workspace-stack-a57cc-bucket
databricks-workspace-stack-a57cc-lambdazipsbucket-5kusfgp8jxa5


#### 29. Find 2nd lowest salary per department using pyspark.

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import *

df_employees = spark.read.format("delta").table("databricks_practicals_new.default.employees_table")

# Define Window Partitioned by Department, Ordered by Salary
window_spec = Window.partitionBy("department").orderBy(col("emp_salary"))

# Apply dense_rank() to rank salaries within each department
df_ranked = df_employees.withColumn("rank", dense_rank().over(window_spec))

# Filter for the 2nd lowest salary (rank = 2)
df_second_lowest = df_ranked.filter(col("rank") == 2).select("department", "emp_name", "emp_salary")

# Show Result
df_second_lowest.show()


+----------+-----------+----------+
|department|   emp_name|emp_salary|
+----------+-----------+----------+
|   Finance|David Clark|   95000.0|
|        HR| Lucy Green|   85000.0|
|        IT|Chris Adams|   77000.0|
| Marketing| Emma White|   68000.0|
|     sales|    Mashood|   98000.0|
+----------+-----------+----------+



#### 30. Write an sql query to delete  a column from table


In [0]:
%sql
ALTER TABLE employees_table DROP COLUMN department;


#### 32. Remove the rows that are having Null vaue and salary is less than 100?

In [0]:
data = [
    ("Ankit", 14300, "Analytics"),
    ("Vikas", 12100, "Analytics"),
    ("Rohit", 7260, "Analytics"),
    ("Mohit", 14000, "IT"),
    ("Mukesh", 7000, "HR"),
    ("Rakesh", 8000, "HR"),
    (None, None, None),
    ("Abc", 10000, None),
    ("Abc1", None, "IT"),
    ("Abc2", -100, "IT")
]

# Define schema
columns = ["ename", "sal", "dname"]

# Create DataFrame
df_emp = spark.createDataFrame(data, columns)

# Show original DataFrame
df_emp.show()

+------+-----+---------+
| ename|  sal|    dname|
+------+-----+---------+
| Ankit|14300|Analytics|
| Vikas|12100|Analytics|
| Rohit| 7260|Analytics|
| Mohit|14000|       IT|
|Mukesh| 7000|       HR|
|Rakesh| 8000|       HR|
|  NULL| NULL|     NULL|
|   Abc|10000|     NULL|
|  Abc1| NULL|       IT|
|  Abc2| -100|       IT|
+------+-----+---------+



In [0]:
# Remove NULL values and salary < 100
df_filtered = df_emp.dropna().filter(col("sal") >= 100)

# Show cleaned DataFrame
df_filtered.show()

+------+-----+---------+
| ename|  sal|    dname|
+------+-----+---------+
| Ankit|14300|Analytics|
| Vikas|12100|Analytics|
| Rohit| 7260|Analytics|
| Mohit|14000|       IT|
|Mukesh| 7000|       HR|
|Rakesh| 8000|       HR|
+------+-----+---------+



#### 33. Data Validation Before Loading to S3 in Databricks

In [0]:
# Schema Validation

expected_schema = ["id", "name"]
actual_schema = sample_df.columns

if set(expected_schema) != set(actual_schema):
    raise ValueError("Schema mismatch! Expected columns:", expected_schema)

In [0]:
# Check for Null Values

null_counts = sample_df.select([col(c).isNull().alias(c) for c in ["id", "name"]]) \
    .agg(*[sum(col(c).cast("int")).alias(c) for c in ["id", "name"]])

# Show the result
null_counts.show()

+---+----+
| id|name|
+---+----+
|  0|   0|
+---+----+



In [0]:
# Remove Duplicates

df_cleaned = sample_df.dropDuplicates()

In [0]:
# Writing to S3 After Validation

df_cleaned.write.mode("overwrite").csv("s3a://abduldbtlearn/processed-data/")